# Some EDA

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Energy = pd.read_csv('../../data/energy_dataset.csv')
Energy = Energy[Energy['total load actual'].notna()]
Weather = pd.read_csv('AllTemps.csv', skipinitialspace=True)

print(list(Energy.keys()))
print(list(Weather.keys()))
print(Weather.shape)
print(Energy.shape)

['time', 'generation biomass', 'generation fossil brown coal/lignite', 'generation fossil coal-derived gas', 'generation fossil gas', 'generation fossil hard coal', 'generation fossil oil', 'generation fossil oil shale', 'generation fossil peat', 'generation geothermal', 'generation hydro pumped storage aggregated', 'generation hydro pumped storage consumption', 'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation marine', 'generation nuclear', 'generation other', 'generation other renewable', 'generation solar', 'generation waste', 'generation wind offshore', 'generation wind onshore', 'forecast solar day ahead', 'forecast wind offshore eday ahead', 'forecast wind onshore day ahead', 'total load forecast', 'total load actual', 'price day ahead', 'price actual']
['Unnamed: 0', 'time', 'tempV', 'tempM', 'tempBi', 'tempBa', 'tempS', 'humV', 'humM', 'humBi', 'humBa', 'humS', 'cldV', 'cldM', 'cldBi', 'cldBa', 'cldS', 'wndV', 'wndM', 'wndBi', 'wndBa'

Made sure that both dataframes are shaped the same

### Let's try to make a dataframe with the weighted averages of the population, and an integer representation of time for input into a Neural Network

In [15]:
pops2017 = {"V": 788000, "M": 3183000, "Bi": 345000, "Ba": 1621000, "S": 689000}
totalPop2017= 788000 + 3183000 + 345000 + 1621000 + 689000
pops2017["V"] /= totalPop2017
pops2017["M"] /= totalPop2017
pops2017["Bi"] /= totalPop2017
pops2017["Ba"] /= totalPop2017
pops2017["S"] /= totalPop2017

def WeightedAverages(Wvar, row, pops):
    return (pops["V"] * row[Wvar + "V"]) + (pops["M"] * row[Wvar + "M"]) + (pops["Bi"] * row[Wvar + "Bi"]) + (pops["Ba"] * row[Wvar + "Ba"]) + (pops["S"] * row[Wvar + "S"])

def IsolateHour(Time):
    Time = Time.split(" ")
    Time = Time[1].split(":")
    return Time[0]

IntTimes = []
TempsAverage = []
HumAvererage = []
CldAverage = []
WndAverage = []
for index, row in Weather.iterrows():
    TempsAverage.append(WeightedAverages("temp", row, pops2017))
    HumAvererage.append(WeightedAverages("hum", row, pops2017))
    CldAverage.append(WeightedAverages("cld", row, pops2017))
    WndAverage.append(WeightedAverages("wnd", row, pops2017))
    IntTimes.append(IsolateHour(row["time"]))


WxAverages = pd.DataFrame({
    "time": np.asarray(IntTimes).astype('float64'),
    "tAve": np.asarray(TempsAverage).astype('float64'),
    "hAve": np.asarray(HumAvererage).astype('float64'),
    "cAve": np.asarray(CldAverage).astype('float64'),
    "wAve": np.asarray(WndAverage).astype('float64')
})


In [16]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# MinMax Scale the numerical features
def scaler_helper(data_column):
    data_column = np.array(data_column).reshape(-1,1)
    scaler = MinMaxScaler()
    scaler.fit(data_column)
    trn_data_col = scaler.transform(data_column)
    return pd.Series(trn_data_col.reshape(-1))
    
# Transform all of the numerical input features to be MinMax Scaled
for col in WxAverages.columns:
    WxAverages[col] = scaler_helper(WxAverages[col])

print(WxAverages.isnull().any())
y = scaler_helper(Energy['total load actual'])
price = scaler_helper(Energy['price actual'])
print(y.isnull().any())
print(price.isnull().any())

time    False
tAve    False
hAve    False
cAve    False
wAve    False
dtype: bool
False
False


### Now to actually do the NN

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics as tfmetrics
from tensorflow.keras.layers import LeakyReLU

def create_model(h1=3, h2=3):
    model = Sequential(name='classifier')

    layers = [
        Dense(5, activation=LeakyReLU(), name='input', kernel_initializer="normal"),
        Dense(h1, activation=LeakyReLU(), name='h1', kernel_initializer="normal"),
        Dense(h2, activation=LeakyReLU(), name='h2', kernel_initializer="normal"),
        Dense(1, activation='tanh', name='out', kernel_initializer="normal")
    ]

    for layer in layers:
        model.add(layer)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=[tfmetrics.RootMeanSquaredError()])

    return model

model = create_model()
pricemodel = create_model()

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest, pricetrain, pricetest = train_test_split(WxAverages, y, price, test_size = 0.1, random_state=0)

hist = model.fit(Xtrain, ytrain, epochs=500, verbose=1)


Epoch 1/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0552 - root_mean_squared_error: 0.2350
Epoch 2/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0275 - root_mean_squared_error: 0.1658
Epoch 3/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0271 - root_mean_squared_error: 0.1647
Epoch 4/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0271 - root_mean_squared_error: 0.1646
Epoch 5/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0270 - root_mean_squared_error: 0.1643
Epoch 6/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0269 - root_mean_squared_error: 0.1641
Epoch 7/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0268 - root_mean_squared_error: 0.1638
Epoch 8/500
986/986 [==============================] - 1s 1ms/step - loss: 0.0268 - root_mean_squared_error: 0.1637
Epoch 9/500
986/986 [==============================] - 1s 1ms/step - los

In [ ]:
histprice = pricemodel.fit(Xtrain, pricetrain, epochs=100, verbose=0)

In [55]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = model.predict(Xtest, verbose=0)
MSE = mean_squared_error(y_true=ytest, y_pred=y_pred)
RMSE = mean_squared_error(y_true=ytest, y_pred=y_pred, squared=False)
r2 = r2_score(y_true=ytest, y_pred=y_pred)
print("Load Scores:")
print('Testing MSE: %.5f' % MSE)
print('Testing RMSE: %.5f' % RMSE)
print("r^2 score: %.5f" % r2)

Load Scores:
Testing MSE: 0.01854
Testing RMSE: 0.13615
r^2 score: 0.53472


In [56]:
price_pred = pricemodel.predict(Xtest, verbose=0)
MSE = mean_squared_error(y_true=pricetest, y_pred=price_pred)
RMSE = mean_squared_error(y_true=pricetest, y_pred=price_pred, squared=False)
r2 = r2_score(y_true=pricetest, y_pred=price_pred)
print("Price Scores:")
print('Testing MSE: %.5f' % MSE)
print('Testing RMSE: %.5f' % RMSE)
print("r^2 score: %.5f" % r2)

Price Scores:
Testing MSE: 0.22387
Testing RMSE: 0.47315
r^2 score: -12.20858
